This code cell performs all the steps above with 1,000 samples per permutation pair and trains the network over 100 epochs, using a 10 fold cross validation, in one go. Therefore, this will take a long time to execute.

In [ ]:
unique_pairs = [str(x)+str(y) for x in range(10) for y in range(10)]
train_counter = 0

(X_train_keras, y_train_keras), (X_test_keras, y_test_keras) = mnist.load_data()

kf = KFold(n_splits=10, shuffle=True, random_state=376483)
kf.get_n_splits(unique_pairs)

unique_pairs_np = np.asarray(unique_pairs)
# Store network performance history and score for each of the training runs.
histories = []
scores = []

# Store accuracies measured in various ways
accuracies_rounded = []
accuracies_floor_ceil = []
accuracies_leeway = []


for train_index, test_index in kf.split(unique_pairs):
    test_set_pairs = unique_pairs_np[test_index]
    train_set_pairs = unique_pairs_np[train_index]
    
    # Sanity checks
    assert(len(test_set_pairs) == 10)
    assert(len(train_set_pairs) == 90)
    for test_set_pair in test_set_pairs:
        assert(test_set_pair not in train_set_pairs)
    
    # If these pass we are good to go with data generation
    X_train = []
    y_train = []

    # Number of samples per permutation (e.g. there are 90 permutations in the train 
    # set so 1000*90 makes 90,000 training samples and 10*1000=10,000 test samples)
    samples_per_permutation = 1000  

    for train_set_pair in train_set_pairs:
        for _ in range(samples_per_permutation):
            rand_i = np.random.choice(np.where(y_train_keras == int(train_set_pair[0]))[0])
            rand_j = np.random.choice(np.where(y_train_keras == int(train_set_pair[1]))[0])
        
            temp_image = np.zeros((28,56), dtype="uint8")
            temp_image[:,:28] = X_train_keras[rand_i]
            temp_image[:,28:] = X_train_keras[rand_j]

            X_train.append(temp_image)
            y_train.append(y_train_keras[rand_i] + y_train_keras[rand_j])
        
    X_test = []
    y_test = []
    
    for test_set_pair in test_set_pairs:
        for _ in range(samples_per_permutation):
            rand_i = np.random.choice(np.where(y_test_keras == int(test_set_pair[0]))[0])
            rand_j = np.random.choice(np.where(y_test_keras == int(test_set_pair[1]))[0])
        
            temp_image = np.zeros((28,56), dtype="uint8")
            temp_image[:,:28] = X_test_keras[rand_i]
            temp_image[:,28:] = X_test_keras[rand_j]
            
            X_test.append(temp_image)
            y_test.append(y_test_keras[rand_i] + y_test_keras[rand_j])
    
    
    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    X_test = np.asarray(X_test)
    y_test = np.asarray(y_test)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
    
    # Some standard preprocessing things here.
    # Reformat the images to use floating point values rather than integers between 0-255
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    # Shuffling
    X_train, y_train = utils.shuffle(X_train, y_train)
    X_test, y_test = utils.shuffle(X_test, y_test)
    
    ######################################################
    # NETWORK SETUP AND TRAINING
    ######################################################
    batch_size = 128
    num_classes = 1               
    epochs = 100
    img_rows, img_cols = np.shape(X_train)[1], np.shape(X_train)[2]
    input_shape = (img_rows, img_cols, 1)

    ######################################################
    # Set up the network model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))  # Default is (3, 3)
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))  # Default is 128
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    # Do not use softmax here, just specify one nueron
    model.add(Dense(num_classes)) 

    ######################################################
    # Choose an optimiser.
    rms = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    sgd = optimizers.SGD(lr=0.0001, decay=1e-5, momentum=0.9, nesterov=True)
    ada = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    ndm = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)

    ######################################################
    # Note: As this is a regression problem, use only mean squared error 
    # or mean absolute error as loss.
    model.compile(loss=losses.mean_squared_error, optimizer=ada)
    
    ## LET'S TRAIN
    histories.append(model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test),
                    verbose=0))
    
    print("RUN %s" % train_counter)
    
    scores.append(model.evaluate(X_test, y_test, verbose=0))
    print(model.evaluate(X_test, y_test, verbose=1))
    
    rounded_correct = 0
    rounded_incorrect = 0
    floor_ceil_correct = 0
    floor_ceil_incorrect = 0
    leeway_correct = 0
    leeway_incorrect = 0

    for i in range(0, len(y_test)):
        prediction = model.predict(X_test[i].reshape(1, X_test[i].shape[0], X_test[i].shape[1], 1))[0][0]

        rounded_prediction = round(prediction)
        floor_prediction = floor(prediction)
        ceiling_prediction = ceil(prediction)

        # Rounded to the nearest integer
        if rounded_prediction == y_test[i]:
            rounded_correct += 1
        else:
            rounded_incorrect += 1

        # Floor or ceiling
        if (floor_prediction == y_test[i]) or (ceiling_prediction == y_test[i]):
            floor_ceil_correct += 1
        else:
            floor_ceil_incorrect += 1

        # Leeway of 1
        abs_difference = abs(rounded_prediction-y_test[i])

        if abs_difference <= 1:
            leeway_correct += 1
        else:
            leeway_incorrect += 1

    accuracies_rounded.append((rounded_correct, rounded_incorrect))
    accuracies_floor_ceil.append((floor_ceil_correct, floor_ceil_incorrect))
    accuracies_leeway.append((leeway_correct, leeway_incorrect))

    print("Correct (rounded): %s, Incorrect (rounded): %s" % (rounded_correct, rounded_incorrect))        
    print("Correct (floor/ceiling): %s, Incorrect (floor/ceiling): %s" % (floor_ceil_correct, floor_ceil_incorrect))
    print("Correct (leeway): %s, Incorrect (leeway): %s" % (leeway_correct, leeway_incorrect))
    
    print("END %s\n" % train_counter)
    
    train_counter += 1

In [ ]:
for history in histories:
    print("Train loss: %s Test loss: %s" % (history.history["loss"][-1], history.history["val_loss"][-1]))

The losses over all epochs plotted for each of the train/test splits in the 10-fold cross validation:

In [ ]:
h_counter = 1
for history in histories:
    score = history.history['val_loss'][-1]
    plt.xlim(0,len(history.history['loss'])-1)
    plt.plot(history.history['loss'], linestyle='--', linewidth=3)
    plt.plot(history.history['val_loss'], linewidth=3)
    plt.title('Loss on Test/Training Set, Split %s' % h_counter)
    plt.ylabel('Loss (MSE)')
    plt.xlabel('Epoch')
    plt.legend(['Training Set', 'Test Set (Loss @ Final Epoch: '+ str("%.2f"%score) +')'], loc='upper right')
    #plt.savefig("/tmp/loss-100-epochs-%s-fold.pdf" % h_counter)
    plt.show()
    plt.close()
    h_counter += 1